In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#initialization of variables

P_thruster = 1250000 #newtons
P_lift = 1400000 #newtons
P_weight = 500000 #newtons

alpha = 5 #degrees

alpha = alpha * np.pi/180

P1 = P_lift*np.sin(alpha) - P_thruster



Hello World
hellow world number 2
3
